## Getting a bit more deeper into Agents

### Conversational ReAct Agents
This agent is designed for use in conversational settings. It incorporates the React framework to determine which tool to use and utilizes memory to remember previous conversation interactions.

In [10]:
from langchain.agents import create_react_agent, load_tools
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv
load_dotenv()

True

In [52]:
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo-0125')

In [53]:
tools = load_tools(["llm-math"],  llm=llm)

In [54]:
memory = ConversationBufferMemory(memory_key="chat_history")

conversational_agent = initialize_agent(
    agent="conversational-react-description", 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,)

In [55]:
output_1=conversational_agent.run("Add 7 to 9 and tell me the result")
output_2=conversational_agent.run("Add 5 to the previous result")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: Add 7 to 9
Observation: Answer: 16
Thought:Do I need to use a tool? No
AI: The result of adding 7 to 9 is 16.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: Add 5 to 16
Observation: Answer: 21
Thought:Do I need to use a tool? No
AI: The result of adding 5 to the previous result of 16 is 21.

> Finished chain.


In [56]:
print(output_1)
print(output_2)

The result of adding 7 to 9 is 16.
The result of adding 5 to the previous result of 16 is 21.


## ReAct Docstore
This agent utilizes the React framework to communicate with a docstore. It requires the availability of a Search tool and a Lookup tool with identical names. The Search tool is used to search for a document, while the Lookup tool looks up a term within the most recently found document.

A `docstore` in Langchain is essentially a storage system for documents. It provides a way for Langchain agents to access and interact with information. Think of it like a library, but instead of books, it holds pieces of text and associated data.

In [57]:
from langchain.agents import initialize_agent, Tool
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer

In [65]:
docstore = DocstoreExplorer(Wikipedia())
tools=[
    Tool(name="Search", func=docstore.search, description="useful for when you need to search wikipedia"),
    Tool(name="Lookup", func=docstore.lookup, description="useful for when you need to lookup a term in wikipedia")]

#intializing the agent
llm = OpenAI(temperature=0)
docstore_agent=initialize_agent(
    tools,
    llm,
    agent="react-docstore",
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=8
)

In [67]:
docstore_agent("Tell me a few key things about Mahatma Gandhi.")



> Entering new AgentExecutor chain...
Thought: I need to search Mahatma Gandhi and find key information about him.
Action: Search[Mahatma Gandhi]
Observation: Mohandas Karamchand Gandhi (ISO: Mōhanadāsa Karamacaṁda Gāṁdhī; 2 October 1869 – 30 January 1948) was an Indian lawyer, anti-colonial nationalist and political ethicist who employed nonviolent resistance to lead the successful campaign for India's independence from British rule. He inspired movements for civil rights and freedom across the world. The honorific Mahātmā (from Sanskrit 'great-souled, venerable'), first applied to him in South Africa in 1914, is now used throughout the world.
Born and raised in a Hindu family in coastal Gujarat, Gandhi trained in the law at the Inner Temple in London, and was called to the bar in June 1891, at the age of 22. After two uncertain years in India, where he was unable to start a successful law practice, he moved to South Africa in 1893 to represent an Indian merchant in a lawsuit. He we

{'input': 'Tell me a few key things about Mahatma Gandhi.',
 'output': 'Mahatma Gandhi was an Indian lawyer, anti-colonial nationalist, and political ethicist who used nonviolent resistance to lead India to independence from British rule. He was known as the Father of the Nation in India and is celebrated worldwide on October 2 as the International Day of Nonviolence.'}

## Self-ask with Search
This agent utilizes the Intermediate Answer tool for self-asking questions.

In [74]:
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()

tools = [
    Tool(
        func=search.run,
        name="Intermediate Answer",
        description="useful for when you need to search the internet for information"
    )

]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="self-ask-with-search",
    verbose=True,
    handle_parsing_errors=True
)

In [76]:
print(agent.invoke("Question: Who was the president of USA when the first Moom landing took place?"))



> Entering new AgentExecutor chain...
 Yes.
Follow up: When did the first Moon landing take place?
Intermediate answer: Intuitive Machines' Nova-C lander, nicknamed Odysseus, is expected to touch down on the lunar surface at 4:24 p.m. ET. If all goes as planned, the lander will be the first American spacecraft on ... Thursday's landing represented the first controlled descent to the lunar surface by a U.S. spacecraft since Apollo 17 in 1972, when NASA's last crewed moon mission landed there with astronauts ... Neil Armstrong (1930-2012) was a U.S. astronaut who became the first human to walk on the moon on July 20, 1969, as part of the Apollo 11 mission. Apollo 11, U.S. spaceflight during which commander Neil Armstrong and lunar module pilot Edwin ("Buzz") Aldrin, Jr., on July 20, 1969, became the first people to land on the Moon and walk the lunar surface. Apollo 11 was the culmination of the Apollo program and a massive national commitment by the United States to beat the Soviet Un

The above code imports necessary modules from the LangChain library, including agents and language models. It sets up a conversational agent with a specific agent configuration called “self-ask-with-search”. The agent utilizes the “Intermediate Answer” tool, which performs Wikipedia searches.